In [88]:
import pandas as pd
import json

from bs4 import BeautifulSoup
import re

In [89]:
dir = 'source/raw_trump_tweets_2.json'

df = pd.read_json(open(dir, 'r'))

In [90]:
df.head()

,created_at,text
0,"datetime.datetime(2018, 9, 26, 10, 57, 30)",Jobless Claims fell to their lowest level in 4...
1,"datetime.datetime(2018, 9, 26, 10, 54, 59)","Consumer confidence hits an 18 year high, clos..."
2,"datetime.datetime(2018, 9, 26, 2, 55, 20)",The Democrats are playing a high level CON GAM...
3,"datetime.datetime(2018, 9, 26, 2, 38, 54)",“These law enforcement people took the law int...
4,"datetime.datetime(2018, 9, 25, 22, 6, 33)",73rd Session of the United Nations General Ass...


In [91]:
# Trying out the lengths of the tweets:
df['pre_clean_len'] = [len(t) for t in df.text]
df['pre_clean_len'][:20]

0      54
1     140
2     139
3     114
4      81
5     140
6     121
7     137
8     119
9      47
10     39
11     23
12    139
13    139
14    109
15    140
16     22
17     23
18    140
19    140
Name: pre_clean_len, dtype: int64

In [92]:
# Trying the cleaning with an example tweet first.
df.text[3191]

"@ricardorossello ......#FakeNews critics are working overtime, but we're getting great marks from the people that truly matter! \n#PRStrong🇵🇷"

In [93]:
example1 = BeautifulSoup(df.text[3191], 'lxml')
print(example1.get_text())

@ricardorossello ......#FakeNews critics are working overtime, but we're getting great marks from the people that truly matter! 
#PRStrong🇵🇷


In [94]:
# Removing special characters, leaving only letters.
re.sub("[^a-zA-Z]", " ", df.text[3191])

' ricardorossello        FakeNews critics are working overtime  but we re getting great marks from the people that truly matter    PRStrong  '

In [95]:
# Actual method for cleaning the text
def tweet_cleaner(text):
    soup = BeautifulSoup(text, 'lxml')
    souped = soup.get_text()

    # Do we want to do something for links etc?
    
    letters_only = re.sub("[^a-zA-Z]", " ", souped)
    lower_case = letters_only.lower()

    words = tok.tokenize(lower_case)
    return (" ".join(words)).strip()

In [96]:
# Testing with small amount of texts
testing = df.text[:10]
test_result = []
for t in testing:
    test_result.append(tweet_cleaner(t))
test_result

['jobless claims fell to their lowest level in years',
 'consumer confidence hits an year high close to breaking the all time record a big jump from last years peop https t co ftdzq lwuu',
 'the democrats are playing a high level con game in their vicious effort to destroy a fine person it is called the https t co ma jxndi',
 'these law enforcement people took the law into their own hands when it came to president trump lindseygrahamsc',
 'rd session of the united nations general assembly unga https t co xjyx rrh f',
 'consumer confidence rose in september notching its highest level in about years the consumer board s index ro https t co ywjgz yu z',
 'remarks by president trump at a luncheon hosted by the secretary general of the united nations https t co wt mdnmdi',
 'remarks by president trump to the rd session of the united nations general assembly https t co oshhko zy https t co fm lsse',
 'rush limbaugh to republicans you can kiss the midterms goodbye if you don t get highly qual

In [97]:
# Running the tweet_cleaner for the big data
print("Cleaning and parsing the tweets...\n")
clean_tweet_texts = []
for i in range(0, len(df)):
    if( (i+1)%1000 == 0 ):
        print(i+1, " tweets has been processed")                                                                  
    clean_tweet_texts.append(tweet_cleaner(df['text'][i]))

Cleaning and parsing the tweets...



/home/venla/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "https://t.co/h43dehf0WV" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/home/venla/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "https://t.co/FXqSWusSTV" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/home/venla/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "https://t.co/qZSEifBNaP" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.

/home/venla/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "https://t.co/E3xvdUGZqa" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/home/venla/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "https://t.co/ue5JEZy85v" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/home/venla/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "https://t.co/yfwdyUHmn3" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.

1000  tweets has been processed


/home/venla/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "https://t.co/4OjDqTMEIx" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/home/venla/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "https://t.co/8lI2lomGkh" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/home/venla/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "https://t.co/6VLQYAlcto" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.

2000  tweets has been processed
3000  tweets has been processed


In [98]:
# Saving clean texts into a dataframe with timestamps
clean_df = pd.DataFrame(clean_tweet_texts,columns=['text'])
clean_df['timestamp'] = df.created_at
clean_df.head()

,text,timestamp
0,jobless claims fell to their lowest level in y...,"datetime.datetime(2018, 9, 26, 10, 57, 30)"
1,consumer confidence hits an year high close to...,"datetime.datetime(2018, 9, 26, 10, 54, 59)"
2,the democrats are playing a high level con gam...,"datetime.datetime(2018, 9, 26, 2, 55, 20)"
3,these law enforcement people took the law into...,"datetime.datetime(2018, 9, 26, 2, 38, 54)"
4,rd session of the united nations general assem...,"datetime.datetime(2018, 9, 25, 22, 6, 33)"
